In [1]:
! pip install transformers torch datasets rouge_score jiwer mlflow

/bin/bash: switchml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `switchml'
/bin/bash: _moduleraw: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `_moduleraw'


In [1]:
import torch
from datasets import load_dataset, load_metric, list_datasets, list_metrics
from transformers import BertTokenizer, EncoderDecoderModel, BasicTokenizer
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

In [2]:
from IPython import embed

In [66]:
import json
import numpy as np
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.pre_tokenizers import Whitespace, Split

output_vocab = {
    "[PAD]": 0,
    "[BOS]": 1,
    "PUT":2,
    "REMOVE":3,
    "USE":4,
    "STOP_USING":5,
    "CHEF_CHECK":6,
    "CHEF_DO":7,
    "MOVE_CONTENTS":8,
}
id_types = {'commands': [2,3,4,5,6,7,8], 'resources':[], 'args': []}
k = len(output_vocab)
with open("res2idx.json", 'r') as f:
    for w,i in json.load(f).items():
        output_vocab[w] = k
        id_types['resources'].append(k)
        k+=1
with open("arg2idx.json", 'r') as f:
    for w,i in json.load(f).items():
#         output_vocab[w] = k
        output_vocab[w.replace('-','_')] = k
        id_types['args'].append(k)
        k+=1

output_vocab = {w:i for i,w in enumerate(output_vocab)}
output_tokenizer = Tokenizer(WordLevel(output_vocab,))
output_tokenizer.pre_tokenizer = Whitespace()

In [67]:
id_types

{'commands': [2, 3, 4, 5, 6, 7, 8],
 'resources': [9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36],
 'args': [37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  153,
  154,
  155,
  156

In [4]:
output_tokenizer.encode_batch(["SERVE MOVE_CONTENTS","SERVE MOVE_CONTENTS PUT"])

[Encoding(num_tokens=2, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=3, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])]

In [5]:
[t.ids for t in _]

[[9, 8], [9, 8, 2]]

In [6]:
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = EncoderDecoderModel.from_encoder_decoder_pretrained('bert-base-uncased', 'bert-base-uncased') # initialize Bert2Bert from pre-trained checkpoints
# # forward
# input_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute", add_special_tokens=True)).unsqueeze(0)  # Batch size 1
# outputs = model(input_ids=input_ids, decoder_input_ids=input_ids)
# # training
# outputs = model(input_ids=input_ids, decoder_input_ids=input_ids, labels=input_ids)
# loss, logits = outputs.loss, outputs.logits
# # save and load from pretrained
# model.save_pretrained("bert2bert")
# model = EncoderDecoderModel.from_pretrained("bert2bert")
# # generation
# generated = model.generate(input_ids, decoder_start_token_id=model.config.decoder.pad_token_id)

In [7]:
csv_file = 'seq2seq_4335716.csv'
input_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
input_tokenizer.bos_token = input_tokenizer.cls_token
input_tokenizer.eos_token = input_tokenizer.sep_token

val_data = load_dataset('csv', data_files=csv_file, split='train[90%:]')
train_data = load_dataset('csv', data_files=csv_file, split='train[:90%]')
print(val_data)
print(train_data)

Using custom data configuration default-b6e1a9e6d71b3baf
Reusing dataset csv (/home/ubuntu/.cache/huggingface/datasets/csv/default-b6e1a9e6d71b3baf/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0)
Using custom data configuration default-b6e1a9e6d71b3baf
Reusing dataset csv (/home/ubuntu/.cache/huggingface/datasets/csv/default-b6e1a9e6d71b3baf/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0)


Dataset({
    features: ['input_seq', 'output_seq'],
    num_rows: 50
})
Dataset({
    features: ['input_seq', 'output_seq'],
    num_rows: 451
})


In [8]:
val_data['output_seq']

['STOP_USING TSPATULA STOVE_MED CHEF_CHECK LBOIL STOVE_MED',
 'USE TSPATULA STOVE_MED PUT IN2e0UIJI STOVE_MED',
 'MOVE_CONTENTS STOVE_MED STOVE_MEDLOW USE TPAN STOVE_MEDLOW STOP_USING TPAN STOVE_MED STOP_USING TSPATULA STOVE_MED CHEF_CHECK LTIME STOVE_MEDLOW',
 'USE TSPATULA STOVE_MEDLOW PUT IzrIHcuDJ STOVE_MEDLOW PUT IrXTUISnn STOVE_MEDLOW',
 'MOVE_CONTENTS STOVE_MED SERVE MOVE_CONTENTS STOVE_MEDLOW SERVE STOP_USING TPAN STOVE_MEDLOW STOP_USING TSPATULA STOVE_MEDLOW PUT Is_lS1dmt SERVE',
 'USE TBOWL COUNTER1 PUT IzrIHcuDJ COUNTER1 PUT IzPYYTv5e COUNTER1 PUT IdCkp3LYx COUNTER1 PUT Is_lS1dmt COUNTER1',
 'MOVE_CONTENTS COUNTER1 OVEN_MED USE TBAKE_DISH OVEN_MED STOP_USING TBOWL COUNTER1 CHEF_CHECK LTIME OVEN_MED',
 'MOVE_CONTENTS OVEN_MED COUNTER1 USE TPAN COUNTER1 STOP_USING TBAKE_DISH OVEN_MED PUT IoskwJgPz COUNTER1 PUT IlGA3C7DK COUNTER1 PUT IraFb0aXd COUNTER1 PUT I6WhEfvid COUNTER1',
 'MOVE_CONTENTS COUNTER1 STOVE_MED MOVE_CONTENTS OVEN_MED STOVE_MED USE TPAN STOVE_MED STOP_USING TPAN

In [9]:
batch_size = 16  # 4 but change to 16 for full training
encoder_max_length = 128
decoder_max_length = 128
device = 'cuda' if torch.cuda.is_available() else 'cpu'


def process_data_to_model_inputs(batch):
    # tokenize the inputs and labels
    tok_params = {
        'padding': 'max_length',
        'truncation': True,
        'max_length': encoder_max_length,
    }
    inputs = input_tokenizer(batch['input_seq'], **tok_params)
    #outputs = output_tokenizer(batch['output_seq'], **tok_params)
    outputs = output_tokenizer.encode_batch(batch['output_seq'])
    # pad
    padded_output = np.zeros((len(outputs),decoder_max_length-1))
    attention_output = np.zeros((len(outputs),decoder_max_length))
    for i,arr in enumerate(outputs):
        padded_output[i,:len(arr.ids)] = arr.ids
        attention_output[i,:len(arr.ids)] = 1
    padded_output=np.concatenate([np.ones((len(outputs),1)),padded_output],axis=1).astype(int)
    attention_output=attention_output.astype(int)
    
    batch['input_ids'] = inputs.input_ids
    batch['attention_mask'] = inputs.attention_mask
    batch['decoder_input_ids'] = padded_output
    batch['decoder_attention_mask'] =attention_output
    batch['labels'] = padded_output

    # because BERT automatically shifts the labels, the labels correspond exactly to `decoder_input_ids`.
    # We have to make sure that the PAD token is ignored
#     batch['labels'] = [
#         [-100 if token == output_tokenizer.pad_token_id else token for token in labels] for labels in batch['labels']
#     ]

    return batch

In [10]:
ed_model.adjust_logits_during_generation??

Object `ed_model.adjust_logits_during_generation` not found.


In [11]:
type(ed_model.decoder)

NameError: name 'ed_model' is not defined

In [11]:
# only use 32 training examples for notebook - COMMENT LINE FOR FULL TRAINING
#train_data = train_data.select(range(32))

train_data = train_data.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
#     remove_columns=['name', 'note'],
)
train_data.set_format(
    type='torch',
    columns=['input_ids', 'attention_mask', 'decoder_input_ids', 'decoder_attention_mask', 'labels'],
)

# only use 16 training examples for notebook - DELETE LINE FOR FULL TRAINING
# val_data = val_data.select(range(16))

val_data = val_data.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
#     remove_columns=['name', 'note'],
)
val_data.set_format(
    type='torch',
    columns=['input_ids', 'attention_mask', 'decoder_input_ids', 'decoder_attention_mask', 'labels'],
)

ed_model = EncoderDecoderModel.from_encoder_decoder_pretrained('bert-base-uncased', 'bert-base-uncased')

# set special tokens
ed_model.config.decoder_start_token_id = 1
ed_model.config.eos_token_id = input_tokenizer.eos_token_id
ed_model.config.pad_token_id = input_tokenizer.pad_token_id

# sensible parameters for beam search
ed_model.config.vocab_size = len(output_vocab)
ed_model.config.max_length = 142
ed_model.config.min_length = 56
ed_model.config.no_repeat_ngram_size = 3
ed_model.config.early_stopping = True
ed_model.config.length_penalty = 2.0
ed_model.config.num_beams = 1



# load wer for validation
wer = load_metric('wer')

def compute_metrics(pred):
    labels_ids = pred.label_ids
#     pred_ids = pred.predictions
#     embed()
    pred_ids = pred.predictions[0].argmax(-1)
    # all unnecessary tokens are removed
    pred_str = output_tokenizer.decode_batch(pred_ids)
    #labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = output_tokenizer.decode_batch(labels_ids)
    #label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    wer_output = wer.compute(predictions=pred_str, references=label_str)

    return {
        'wer': round(wer_output, 4),
    }


# set training arguments - these params are not really tuned, feel free to change
# training_args = Seq2SeqTrainingArguments(
#     output_dir='./',
#     evaluation_strategy='steps',
#     per_device_train_batch_size=batch_size,
#     per_device_eval_batch_size=batch_size,
#     predict_with_generate=True,
#     logging_steps=500,  # 2 or set to 1000 for full training
#     save_steps=500,  # 16 or set to 500 for full training
#     eval_steps=500,  # 4 or set to 8000 for full training
#     warmup_steps=500,  # 1 or set to 2000 for full training
#     max_steps=2500,  # 16 or comment for full training
#     overwrite_output_dir=True,
#     save_total_limit=3,
#     fp16=torch.cuda.is_available(),
# )

training_args = Seq2SeqTrainingArguments(
    output_dir='./',
    evaluation_strategy='steps',
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
#     predict_with_generate=True, #TODO: is this necessary ?
    logging_steps=50,  # 2 or set to 1000 for full training
    save_steps=50,  # 16 or set to 500 for full training
    eval_steps=50,  # 4 or set to 8000 for full training
    warmup_steps=50,  # 1 or set to 2000 for full training
    max_steps=850,  # 16 or comment for full training
    overwrite_output_dir=True,
    save_total_limit=3,
    fp16=torch.cuda.is_available(),
)

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=ed_model,
    tokenizer=input_tokenizer,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=val_data,
)
trainer.train()

Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/csv/default-b6e1a9e6d71b3baf/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0/cache-dbb95374d2b40c61.arrow
Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/csv/default-b6e1a9e6d71b3baf/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0/cache-ebeb16ec743a2cf7.arrow
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model 

Step,Training Loss,Validation Loss,Wer,Runtime,Samples Per Second
50,5.944000,0.813652,0.096900,2.097400,23.839000
100,0.588500,0.402868,0.096900,2.065600,24.206000
150,0.364700,0.369712,0.089500,2.046900,24.427000
200,0.351500,0.367194,0.089800,2.101500,23.792000
250,0.346200,0.360239,0.086600,2.056100,24.318000
300,0.334100,0.356488,0.086900,2.054400,24.338000
350,0.313800,0.350978,0.089400,2.088500,23.941000
400,0.309300,0.345599,0.088900,2.103000,23.776000
450,0.282300,0.343013,0.086100,2.057900,24.296000
500,0.274300,0.347001,0.086900,2.078300,24.058000


TrainOutput(global_step=850, training_loss=0.6350265760982737, metrics={'train_runtime': 412.0624, 'train_samples_per_second': 2.063, 'total_flos': 5024080977527808.0, 'epoch': 29.31, 'init_mem_cpu_alloc_delta': 6421765, 'init_mem_gpu_alloc_delta': 991727104, 'init_mem_cpu_peaked_delta': 18306, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 3057007, 'train_mem_gpu_alloc_delta': 3117464576, 'train_mem_cpu_peaked_delta': 803123979, 'train_mem_gpu_peaked_delta': 4844791296})

In [ ]:
ed_model.config.decoder.

In [55]:
inputs

{'input_ids': tensor([[ 101, 5660, 7975, 2127, 2053, 2936, 5061,  102]], device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [68]:
txt = "Cook chicken until no longer pink"
inputs = input_tokenizer(txt, return_tensors="pt").to(device)
ret = ed_model.forward(decoder_input_ids=inputs['input_ids'], **inputs)
ret

Seq2SeqLMOutput(loss=None, logits=tensor([[[ 9.8588,  6.5779, 10.5644,  ...,  0.1462,  2.5806, -3.6224],
         [ 9.8574,  6.5773, 10.5578,  ...,  0.1454,  2.5822, -3.6268],
         [ 9.8651,  6.5815, 10.5622,  ...,  0.1444,  2.5864, -3.6146],
         ...,
         [ 9.8603,  6.5730, 10.5536,  ...,  0.1404,  2.5811, -3.6490],
         [ 9.8525,  6.5716, 10.5405,  ...,  0.1456,  2.5879, -3.6464],
         [ 9.7814,  7.7578, 10.6245,  ...,  0.7323,  0.2888, -1.6048]]],
       device='cuda:0', grad_fn=<AddBackward0>), past_key_values=((tensor([[[[ 1.2599,  0.1719,  0.1596,  ...,  0.4940, -0.3351,  0.9681],
          [ 1.9662,  0.1820,  0.3237,  ..., -0.6993, -0.3477, -1.0078],
          [-0.3461, -0.9077,  0.7523,  ..., -0.1725, -0.1261, -1.1778],
          ...,
          [-0.2509,  0.7811,  0.9580,  ..., -0.4399, -0.3374, -1.8014],
          [-0.1176, -0.2355,  0.6322,  ...,  0.1092,  0.0225,  0.2288],
          [-0.6247,  1.0394, -0.0855,  ..., -0.6001,  0.2802,  0.6803]],

        

In [75]:
ret.keys()

odict_keys(['logits', 'past_key_values', 'encoder_last_hidden_state'])

In [96]:
pred_ids = [1] + \
    ret['logits'].argmax(-2).to('cpu').numpy()[0]

In [97]:
output_tokenizer.decode(pred_ids)

'REMOVE MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS REMOVE MOVE_CONTENTS [BOS] MOVE_CONTENTS REMOVE MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_CONTENTS MOVE_C

In [14]:
from types import MethodType
def adjust(self, logits, **kwargs):
    print ("boom")
    return logits
ed_model.decoder.adjust_logits_during_generation = MethodType(adjust, ed_model.decoder)

NameError: name 'ed_model' is not defined

# Backup

In [247]:
ed_model.save_pretrained("recipe_bert_seq2seq")
!aws s3 cp --recursive recipe_bert_seq2seq s3://uatt-data/uri/
!rm -rf recipe_bert_seq2seq

/bin/bash: switchml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `switchml'
/bin/bash: _moduleraw: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `_moduleraw'
upload: recipe_bert_seq2seq/config.json to s3://uatt-data/uri/config.json
upload: recipe_bert_seq2seq/pytorch_model.bin to s3://uatt-data/uri/pytorch_model.bin
/bin/bash: switchml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `switchml'
/bin/bash: _moduleraw: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `_moduleraw'


# Tests

In [93]:
input_text="Cook chicken until golden brown ."
input_ids = torch.tensor(input_tokenizer.encode(input_text, add_special_tokens=True)).unsqueeze(0).to(device)  # Batch size 1
generated = ed_model.generate(input_ids, decoder_start_token_id=1)

In [95]:
output_ids = generated.to('cpu').numpy().reshape(-1)
output_tokenizer.decode(output_ids)

'[BOS] MOVE_CONTENTS COUNTER1 STOVE_MED STOVE_MED STOVE_MED STOVE_MEDLOW STOVE_MED STOVE_MED PUT STOVE_MED STOVE_MED [PAD] [PAD] [PAD] PUT STOVE_MED PUT PUT STOVE_MED [PAD] CHEF_CHECK STOVE_MED STOVE_MED CHEF_CHECK STOVE_MED [PAD] PUT [PAD] [PAD] CHEF_CHECK [PAD] [PAD] STOVE_MED STOVE_MED TPAN STOVE_MED STOVE_MED LTIME STOVE_MED STOVE_MED STOP_USING [PAD] [PAD] STOP_USING STOVE_MED [PAD] STOVE_MED [PAD] LTIME STOVE_MED [PAD] STOP_USING [PAD] CHEF_CHECK PUT STOVE_MED CHEF_CHECK [PAD] CHEF_CHECK CHEF_CHECK [PAD] PUT PUT [PAD] CHEF_CHECK STOVE_MEDLOW [PAD] [PAD] LTIME [PAD] [PAD] COUNTER1 STOVE_MED [PAD] COUNTER1 [PAD] [PAD] STOVE_MEDLOW STOVE_MED [PAD] STOVE_MEDLOW [PAD] CHEF_CHECK LTIME STOVE_MED PUT [PAD] PUT CHEF_CHECK STOVE_MED PUT CHEF_CHECK [PAD] STOVE_MED PUT STOVE_MEDLOW [PAD] PUT STOVE_MEDLOW STOVE_MED PUT LTIME [PAD] CHEF_CHECK TPAN [PAD] [PAD] SERVE [PAD] [PAD] MOVE_CONTENTS [PAD] [PAD] USE [PAD] [PAD] STOVE_LOW STOVE_MED STOVE_MED STOVE_MEDHI [PAD] [PAD] TPAN STOVE_MED [PAD] 

In [250]:
!aws s3 cp transformers-recipe.ipynb s3://uatt-notebooks/

/bin/bash: switchml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `switchml'
/bin/bash: _moduleraw: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `_moduleraw'
upload: ./transformers-recipe.ipynb to s3://uatt-notebooks/transformers-recipe.ipynb


# Beam search override

In [35]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from typing import *
import warnings
from transformers.generation_utils import *

class GorenSampleMixin:
    def greedy_search(
        self,
        input_ids: torch.LongTensor,
        logits_processor = None,
        stopping_criteria = None,
        max_length = None,
        pad_token_id = None,
        eos_token_id = None,
        output_attentions = None,
        output_hidden_states = None,
        output_scores = None,
        return_dict_in_generate = None,
        synced_gpus= None,
        **model_kwargs,
    ):
        r"""
        Generates sequences for models with a language modeling head using greedy decoding.

        Parameters:

            input_ids (:obj:`torch.LongTensor` of shape :obj:`(batch_size, sequence_length)`, `optional`):
                The sequence used as a prompt for the generation. If :obj:`None` the method initializes it as an empty
                :obj:`torch.LongTensor` of shape :obj:`(1,)`.
            logits_processor (:obj:`LogitsProcessorList`, `optional`):
                An instance of :class:`~transformers.LogitsProcessorList`. List of instances of class derived from
                :class:`~transformers.LogitsProcessor` used to modify the prediction scores of the language modeling
                head applied at each generation step.
            stopping_criteria (:obj:`StoppingCriteriaList`, `optional`):
                An instance of :class:`~transformers.StoppingCriteriaList`. List of instances of class derived from
                :class:`~transformers.StoppingCriteria` used to tell if the generation loop should stop.

            max_length (:obj:`int`, `optional`, defaults to 20):
                **DEPRECATED**. Use :obj:`logits_processor` or :obj:`stopping_criteria` directly to cap the number of
                generated tokens. The maximum length of the sequence to be generated.
            pad_token_id (:obj:`int`, `optional`):
                The id of the `padding` token.
            eos_token_id (:obj:`int`, `optional`):
                The id of the `end-of-sequence` token.
            output_attentions (:obj:`bool`, `optional`, defaults to `False`):
                Whether or not to return the attentions tensors of all attention layers. See ``attentions`` under
                returned tensors for more details.
            output_hidden_states (:obj:`bool`, `optional`, defaults to `False`):
                Whether or not to return trhe hidden states of all layers. See ``hidden_states`` under returned tensors
                for more details.
            output_scores (:obj:`bool`, `optional`, defaults to `False`):
                Whether or not to return the prediction scores. See ``scores`` under returned tensors for more details.
            return_dict_in_generate (:obj:`bool`, `optional`, defaults to `False`):
                Whether or not to return a :class:`~transformers.file_utils.ModelOutput` instead of a plain tuple.
            synced_gpus (:obj:`bool`, `optional`, defaults to :obj:`False`):
                Whether to continue running the while loop until max_length (needed for ZeRO stage 3)
            model_kwargs:
                Additional model specific keyword arguments will be forwarded to the :obj:`forward` function of the
                model. If model is an encoder-decoder model the kwargs should include :obj:`encoder_outputs`.

        Return:
            :class:`~transformers.generation_utils.GreedySearchDecoderOnlyOutput`,
            :class:`~transformers.generation_utils.GreedySearchEncoderDecoderOutput` or obj:`torch.LongTensor`: A
            :obj:`torch.LongTensor` containing the generated tokens (default behaviour) or a
            :class:`~transformers.generation_utils.GreedySearchDecoderOnlyOutput` if
            ``model.config.is_encoder_decoder=False`` and ``return_dict_in_generate=True`` or a
            :class:`~transformers.generation_utils.GreedySearchEncoderDecoderOutput` if
            ``model.config.is_encoder_decoder=True``.

        Examples::

            >>> from transformers import (
            ... AutoTokenizer,
            ... AutoModelForCausalLM,
            ... LogitsProcessorList,
            ... MinLengthLogitsProcessor,
            ... )

            >>> tokenizer = AutoTokenizer.from_pretrained("gpt2")
            >>> model = AutoModelForCausalLM.from_pretrained("gpt2")

            >>> # set pad_token_id to eos_token_id because GPT2 does not have a EOS token
            >>> model.config.pad_token_id = model.config.eos_token_id

            >>> input_prompt = "Today is a beautiful day, and"
            >>> input_ids = tokenizer(input_prompt, return_tensors="pt").input_ids

            >>> # instantiate logits processors
            >>> logits_processor = LogitsProcessorList([
            ...     MinLengthLogitsProcessor(15, eos_token_id=model.config.eos_token_id),
            ... ])

            >>> outputs = model.greedy_search(input_ids, logits_processor=logits_processor)

            >>> print("Generated:", tokenizer.batch_decode(outputs, skip_special_tokens=True))
        """
        # init values
        logits_processor = logits_processor if logits_processor is not None else LogitsProcessorList()
        stopping_criteria = stopping_criteria if stopping_criteria is not None else StoppingCriteriaList()
        if max_length is not None:
            warnings.warn(
                "`max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList(MaxLengthCriteria(max_length=max_length))` instead.",
                UserWarning,
            )
            stopping_criteria = validate_stopping_criteria(stopping_criteria, max_length)
        pad_token_id = pad_token_id if pad_token_id is not None else self.config.pad_token_id
        eos_token_id = eos_token_id if eos_token_id is not None else self.config.eos_token_id
        output_scores = output_scores if output_scores is not None else self.config.output_scores
        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        )
        return_dict_in_generate = (
            return_dict_in_generate if return_dict_in_generate is not None else self.config.return_dict_in_generate
        )

        # init attention / hidden states / scores tuples
        scores = () if (return_dict_in_generate and output_scores) else None
        decoder_attentions = () if (return_dict_in_generate and output_attentions) else None
        cross_attentions = () if (return_dict_in_generate and output_attentions) else None
        decoder_hidden_states = () if (return_dict_in_generate and output_hidden_states) else None

        # if model is an encoder-decoder, retrieve encoder attention weights and hidden states
        if return_dict_in_generate and self.config.is_encoder_decoder:
            encoder_attentions = model_kwargs["encoder_outputs"].get("attentions") if output_attentions else None
            encoder_hidden_states = (
                model_kwargs["encoder_outputs"].get("hidden_states") if output_hidden_states else None
            )

        # keep track of which sequences are already finished
        unfinished_sequences = input_ids.new(input_ids.shape[0]).fill_(1)
        cur_len = input_ids.shape[-1]

        this_peer_finished = False  # used by synced_gpus only
        while True:

            if synced_gpus:
                # Under synced_gpus the `forward` call must continue until all gpus complete their sequence.
                # The following logic allows an early break if all peers finished generating their sequence
                this_peer_finished_flag = torch.tensor(0.0 if this_peer_finished else 1.0).to(input_ids.device)
                # send 0.0 if we finished, 1.0 otherwise
                dist.all_reduce(this_peer_finished_flag, op=dist.ReduceOp.SUM)
                # did all peers finish? the reduced sum will be 0.0 then
                if this_peer_finished_flag.item() == 0.0:
                    break

            # prepare model inputs
            model_inputs = self.prepare_inputs_for_generation(input_ids, **model_kwargs)

            # forward pass to get next token
            outputs = self(
                **model_inputs,
                return_dict=True,
                output_attentions=output_attentions,
                output_hidden_states=output_hidden_states,
            )

            if synced_gpus and this_peer_finished:
                cur_len = cur_len + 1
                continue  # don't waste resources running the code we don't need

            next_token_logits = outputs.logits[:, -1, :]

            # Store scores, attentions and hidden_states when required
            if return_dict_in_generate:
                if output_scores:
                    scores += (next_token_logits,)
                if output_attentions:
                    decoder_attentions += (
                        (outputs.decoder_attentions,) if self.config.is_encoder_decoder else (outputs.attentions,)
                    )
                    if self.config.is_encoder_decoder:
                        cross_attentions += (outputs.cross_attentions,)

                if output_hidden_states:
                    decoder_hidden_states += (
                        (outputs.decoder_hidden_states,)
                        if self.config.is_encoder_decoder
                        else (outputs.hidden_states,)
                    )

            # pre-process distribution
            next_tokens_scores = logits_processor(input_ids, next_token_logits)

            # argmax
            next_tokens = torch.argmax(next_tokens_scores, dim=-1)

            # finished sentences should have their next token be a padding token
            if eos_token_id is not None:
                assert pad_token_id is not None, "If eos_token_id is defined, make sure that pad_token_id is defined."
                next_tokens = next_tokens * unfinished_sequences + pad_token_id * (1 - unfinished_sequences)

            # update generated ids, model inputs, and length for next step
            input_ids = torch.cat([input_ids, next_tokens[:, None]], dim=-1)
            model_kwargs = self._update_model_kwargs_for_generation(
                outputs, model_kwargs, is_encoder_decoder=self.config.is_encoder_decoder
            )
            cur_len = cur_len + 1

            # if eos_token was found in one sentence, set sentence to finished
            if eos_token_id is not None:
                unfinished_sequences = unfinished_sequences.mul((next_tokens != eos_token_id).long())

            # stop when each sentence is finished, or if we exceed the maximum length
            if unfinished_sequences.max() == 0 or stopping_criteria(input_ids, scores):
                if not synced_gpus:
                    break
                else:
                    this_peer_finished = True

        if return_dict_in_generate:
            if self.config.is_encoder_decoder:
                return GreedySearchEncoderDecoderOutput(
                    sequences=input_ids,
                    scores=scores,
                    encoder_attentions=encoder_attentions,
                    encoder_hidden_states=encoder_hidden_states,
                    decoder_attentions=decoder_attentions,
                    cross_attentions=cross_attentions,
                    decoder_hidden_states=decoder_hidden_states,
                )
            else:
                return GreedySearchDecoderOnlyOutput(
                    sequences=input_ids,
                    scores=scores,
                    attentions=decoder_attentions,
                    hidden_states=decoder_hidden_states,
                )
        else:
            return input_ids

class GorenGPT2LMHeadModel(GorenSampleMixin, GPT2LMHeadModel):
    pass

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GorenGPT2LMHeadModel.from_pretrained('gpt2')

inputs = tokenizer("Hello, my dog is cute and ", return_tensors="pt")
# this should work...
generation_output = model.generate(**inputs)  
# this should print "cool" and fail the assert...
# generation_output = model.generate(**inputs, num_beams=2, do_sample=True)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/home/ubuntu/anaconda3/envs/pytorch_latest_p37/lib/python3.7/site-packages/ipykernel/__main__.py:103: UserWarning: `max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList(MaxLengthCriteria(max_length=max_length))` instead.


TypeError: 'NoneType' object is not callable

In [25]:
generation_output

tensor([[15496,    11,   616,  3290,   318, 13779,   290,   220, 17479,    13,
           314,  1101,   407,  1654,   611,   673,   338,   257,   922,  3290]])

In [40]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

inputs = tokenizer("Hello, my dog is cute and ", return_tensors="pt")
# this should work...
model.generate(**inputs, return_dict_in_generate=True)  

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


GreedySearchDecoderOnlyOutput(sequences=tensor([[15496,    11,   616,  3290,   318, 13779,   290,   220, 17479,    13,
           314,  1101,   407,  1654,   611,   673,   338,   257,   922,  3290]]), scores=None, attentions=None, hidden_states=None)

In [48]:
d = model(**inputs)
type(d)

transformers.modeling_outputs.CausalLMOutputWithCrossAttentions

In [56]:
d.logits.argmax(-1)

tensor([[   11,   314,  1438,   318,   257,    13,   314, 17479]])

In [53]:
inputs["input_ids"].shape

torch.Size([1, 8])

In [80]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import BartTokenizer, Ba
from typing import *
import warnings
from transformers.generation_utils import *

class MyAdjustMixin:
    def adjust_logits_during_generation(self, logits, **kwargs):
        print ("boom")
        return logits


class GorenGPT2LMHeadModel(MyAdjustMixin, GPT2LMHeadModel):
    pass

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GorenGPT2LMHeadModel.from_pretrained('gpt2')

inputs = tokenizer("Hello, my dog is cute and ", return_tensors="pt")
# this should work...
generation_output = model.generate(**inputs)  
# this should print "cool" and fail the assert...
# generation_output = model.generate(**inputs, num_beams=2, do_sample=True)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [81]:
generation_output

tensor([[15496,    11,   616,  3290,   318, 13779,   290,   220, 17479,    13,
           314,  1101,   407,  1654,   611,   673,   338,   257,   922,  3290]])